<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод:</a></span></li><li><span><a href="#-Комментарий-ревьюера" data-toc-modified-id="-Комментарий-ревьюера-6"><span class="toc-item-num">6&nbsp;&nbsp;</span> Комментарий ревьюера</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE
from numpy import linalg
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
df.head(15)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Наши текущие данные - исключительно числовые, даже пол указан как ноль или единица 

(предположу что 1-м, 0-ж, это из моих вузовских знаний)

Цифры семейного положения легче было бы принимать как кол-во детей, но скорее всего 0 - нет пары, 1 - женат/замужем а остальное кол-во детей, но думаю это не так важно :)

присутсвуют как int так и float, лучше переведем всё во float дабы матрицы были по красивее, так же проведем стандартную проверку на пропуски

На дубли проверять нет смысла

In [6]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

пропусков нет :)

In [7]:
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null int64
Зарплата             5000 non-null int64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


In [9]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


### Вывод: 
Мы открыли и прочитали данные, посмотрели на их корректность и поправили типы данных столбцов "Возраст" и "Зарплата"

## Умножение матриц

Посмотрим изменится ли r2 при умножении признаков на обратимую матрицу

In [10]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

In [11]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
model = LinearRegression()
model.fit(features, target)
model.coef_

array([ 7.92580563e-03,  3.57083050e-02, -1.70081903e-07, -1.35676627e-02])

array([ 7.92580563e-03,  3.57083050e-02, -1.70081903e-07, -1.35676627e-02])

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$
Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$
Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


**Вопрос**: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ**: не изменится.

Заменим матрицу $X$ на матрицу $Z$:

$$
Z = XP \qquad (2.1)
$$
где $P$ - обратимая матрица с некими значениями, на которую может быть умножена матрица $X$

**Обоснование**:
Заменим X на Z и вычислим, чему будет равно предсказание и вектор весов:

$$
a_1 = Zw_1 \qquad
$$$$
w_1 = (Z^T Z)^{-1} Z^T y \qquad 
$$


Скрестим эти два уравнения
$$
a_1 = Z (Z^T Z)^{-1} Z^T y \qquad
$$

Заменим все $Z$ правой частью уравнения на $XP$

$$
a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y \qquad
$$

Для следующего шага понадобится следующее свойсво обратной матрицы:

$$
(AB)^{-1} = B^{-1} A^{-1}
$$
Раскроем $((XP)^T (XP))^{-1}$ в два шага:

$$
a_1 = XP ((XP)^T (XP))^{-1} (XP)^T y = XP(XP)^{-1}((XP)^T)^{-1} (XP)^T y = XPP^{-1}X^{-1}((XP)^T)^{-1} (XP)^T y  \qquad (2.6)
$$
Умножение приведет к $PP^{-1} = E$. Для следуюзего этапа воспользуемся свойством транспорнированной матрицы:

$$
(AB)^T = B^T A^T
$$
Умножение на единичную матрицу ничего не меняет. Раскроем $((XP)^T)^{-1} (XP)^T$ в три шага:


$$
a_1 = XEX^{-1}((XP)^T)^{-1} (XP)^T y = XX^{-1}(P^T X^T)^{-1} P^T X^T y = XX^{-1} (X^T)^{-1} (P^T)^{-1}  P^T X^T y   \qquad (2.7)
$$
Умножение приведет к $(P^T)^{-1}  P^T = E$. Посмотрим, что осталось от уравнения 

$$
a_1 = XX^{-1} (X^T)^{-1} E X^T y = X(X^T X)^{-1} X^T y = Xw = a  \qquad 
$$
и у нас получилось, что значение предсказания $a$ не меняется, если умножать матрицу признаков на обратимую матрицу

### Вывод:

Как видно, значение предсказания $a$ не меняется, если умножать матрицу признаков на обратимую матрицу

## Алгоритм преобразования

**Алгоритм**

Для защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу $Y$, которая будет генерироваться случайным образом.

Этапы алгоритма:

Составление матрицы $Y$.

Проверка матрицы на обратимость. Вычисление детерминанта матрицы $Y$.

Получение матрицы преобразованных признаков $Z = XY$.

Применение алгоритма на преобразованных признаках $Z$.

Матрица $Y$ должна иметь необходимую размерность $(nxn)$, где n - количество признаков для регрессии. Таким образом матрица $Z$ будет иметь туже размерность, что и матрица $X$. Обратная матрица $Y$ существует только для квадратных невырожденных матриц (определитель которых не равен нулю).



$
X = \begin{pmatrix}
1 | 2 \\
2 | 3 \\
4 | 5  
\end{pmatrix}
\qquad 
Y = \begin{pmatrix}
1 | 0 \\
2 | 3 \\  
\end{pmatrix}
\qquad det  Y = 3 $

Найдем значние Z:

$
Z = \begin{pmatrix}
1 | 2 \\
2 | 3 \\
4 | 5  
\end{pmatrix} \begin{pmatrix}
1 | 0 \\
2 | 3 \\  
\end{pmatrix} = \begin{pmatrix}
1*1+2*2 | 1*0+2*3  \\
2*1+3*2 | 2*0+3*3  \\
4*1+5*2 | 4*0+5*3   
\end{pmatrix} = \begin{pmatrix}
5 | 6 \\
8 | 9 \\
14 | 15  
\end{pmatrix}
$

После этого добавляем нулевой столбец и вводим данные в линейную регрессию.

## Проверка алгоритма

Проведем исследование модели по двум направлениям:

**1** Исследуем качество модели без преобразования.

      1.1 С исходными признаками
  
      1.2 С отмасштабированными признаками
  
  
**2** Исследуем качество модели с преобразованием.

      2.1 С исходными признаками
  
      2.2 С отмасштабированными признаками
  
  


In [12]:
# разделим данные на обучающие и тестовые
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [13]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_origin_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_origin_data)

w-vector coef [ 1.79258370e-02  3.57228278e-02 -5.46002729e-07 -1.26186599e-02]
R2 = 0.4352275684083349


Посмотрим на модель при масштабировании признаков 

In [14]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_origin_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_origin_data_scaled)

R2 = 0.43522756840833376


ничего не изменилось, небольшая разность результата связанна с точностью хранения данных в питоне

напишем функцию преобразования матрицы признаков

In [15]:
def cipher_features(features):
    crypted_features = features
    n = features.shape[1]
    np.random.seed(7072020)
    cipher_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(cipher_matrix)
    while det == 0:
        np.random.seed(7072021)
        cipher_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(cipher_matrix)
    crypted_features = crypted_features @ cipher_matrix
    return crypted_features, cipher_matrix

In [16]:
display(features.head())
features, cipher_matrix = cipher_features(features)
display(features.head())
cipher_matrix

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600,1
1,0,46,38000,1
2,0,29,21000,0
3,0,21,41700,2
4,1,28,26100,0


,0,1,2,3
0,397184,49981,99373,248097
1,304422,38421,76185,190101
2,168261,21261,42116,105058
3,333805,41903,83486,208560
4,209059,26357,52320,130562


array([[7, 5, 8, 6],
       [9, 9, 4, 2],
       [8, 1, 2, 5],
       [8, 7, 1, 9]])

Данные потеряли свой прежний вид, теперь посчитаем метрику r2

In [17]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=7072020)

In [18]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_cipher_data)

w-vector coef [ 0.00319167  0.00168011  0.00069429 -0.0057204 ]
R2 = 0.4184108158256896


In [19]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_cipher_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_cipher_data_scaled)

R2 = 0.41841081582565687


In [20]:
result = pd.DataFrame(data= [R2_LR_origin_data_scaled,
                      R2_LR_origin_data,
                      R2_LR_cipher_data,
                      R2_LR_cipher_data_scaled], 
                     columns=['R2'], 
                     index=['Линейная регрессия',
                            'Линейная регрессия c масштабом',
                            'Линейная регрессия на преобразованных признаках',
                            'Линейная регрессия на преобразованных признаках c масштабом',])
result

,R2
Линейная регрессия,0.435228
Линейная регрессия c масштабом,0.435228
Линейная регрессия на преобразованных признаках,0.418411
Линейная регрессия на преобразованных признаках c масштабом,0.418411


## Вывод:

В этой работе мы сделали следующее:
    
    1) Выяснили, что качество модели не изменяется от умножения признаков на обратную матрицу и объяснили почему
    
    2) Предложили и испробовали алгоритм кодирования

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования